# Get Artifacts

In [18]:
import sys
import torch

import os; os.environ['CUDA_VISIBLE_DEVICES']='0'
torch.cuda.set_device('cuda:0')
sys.path.append("/home/hamel/hamel_fork/edu/mlops-001/lesson3/")


# - Try something else

In [17]:
#|export
import wandb
from fastai.vision.all import *
import params
import pandas as pd
from pathlib import Path
from train import prepare_df

In [9]:
#|export
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_data_split(model_registry_artifact):
    "Get the validation and test dataset from the artifacts associated with the model from the registry."
    parent_run_artifacts = model_registry_artifact.logged_by().used_artifacts()
    datasets_used = [a for a in parent_run_artifacts if a.type == 'split_data']
    assert len(datasets_used) == 1, f'Only expected one input artifact with type == `split_data`, but found {len(datasets_used)}'
    data_artifact = datasets_used[0]
    artifact_dir = Path(data_artifact.download())
    return prepare_df(artifact_dir), prepare_df(artifact_dir, is_test=True)
    

def get_eval_artifacts(run, art_name='av-team/model-registry/BDD Semantic Segmentation:staging', art_type='model'):
    "Return the learner and the data split with the holdout set."
    artifact = run.use_artifact(art_name, type=art_type)
    artifact_dir = Path(artifact.download())
    model = load_learner(artifact_dir/'fastai_model.pkl')
    model.cuda()
    train_df, test_df = get_data_split(artifact)
   
    return model, train_df, test_df

In [10]:
#|export
run = wandb.init(project=params.WANDB_PROJECT, 
                 entity=params.ENTITY, 
                 job_type="evaluation")

fastai_model, train_df, test_df = get_eval_artifacts(run)

wandb: Currently logged in as: hamelsmu (av-team). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact BDD Semantic Segmentation:staging, 237.50MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact bdd_simple_1k_split:v4, 813.25MB. 4010 files... 
wandb:   4010 of 4010 files downloaded.  
Done. 0:0:0.3


In [11]:
assert torch.cuda.is_available()
assert next(fastai_model.model.parameters()).is_cuda

In [12]:
train_df.Stage.value_counts()

train    800
valid    100
Name: Stage, dtype: int64

In [13]:
test_df.Stage.value_counts()

test    100
Name: Stage, dtype: int64

In [16]:
train_df.head()

,File_Name,Stage,is_valid,image_fname,label_fname
0,99ab7304-32307eea.jpg,train,False,artifacts/bdd_simple_1k_split:v4/images/99ab7304-32307eea.jpg,artifacts/bdd_simple_1k_split:v4/labels/99ab7304-32307eea_mask.png
1,a9391f8f-83c00a81.jpg,train,False,artifacts/bdd_simple_1k_split:v4/images/a9391f8f-83c00a81.jpg,artifacts/bdd_simple_1k_split:v4/labels/a9391f8f-83c00a81_mask.png
2,c2841851-644a8cfe.jpg,train,False,artifacts/bdd_simple_1k_split:v4/images/c2841851-644a8cfe.jpg,artifacts/bdd_simple_1k_split:v4/labels/c2841851-644a8cfe_mask.png
3,28f73d40-e4720001.jpg,train,False,artifacts/bdd_simple_1k_split:v4/images/28f73d40-e4720001.jpg,artifacts/bdd_simple_1k_split:v4/labels/28f73d40-e4720001_mask.png
4,9b2db8cb-00000000.jpg,train,False,artifacts/bdd_simple_1k_split:v4/images/9b2db8cb-00000000.jpg,artifacts/bdd_simple_1k_split:v4/labels/9b2db8cb-00000000_mask.png


In [14]:
#|export 
valid_dls = fastai_model.dls.test_dl(train_df[train_df.Stage == 'valid'], device=torch.device('cuda:0'))
val_metrics = fastai_model.validate(dl=valid_dls)

/home/hamel/hamel_fork/edu/mlops-001/lesson3/utils.py:88: RuntimeWarning: Mean of empty slice
  return np.nanmean(binary_iou_scores)


KeyError: 'Exception occured in `Recorder` when calling event `after_validate`:\n\t0'

In [ ]:
test_dls = fastai_model.dls.test_dl(train_df[train_df.Stage == 'test'], device=torch.device('cuda:0'))
test_metrics = fastai_model.validate(dl=valid_dls)

In [ ]:
from nbdev.export import nb_export
nb_export('evaluate.ipynb', lib_path='.', name='evaluate')

In [15]:
%debug

> /home/hamel/hamel_fork/edu/mlops-001/lesson3/utils.py(94)value()
     92     def value(self):
     93         c=CLASS_INDEX[self.nm]
---> 94         return self.inter[c]/(self.union[c]-self.inter[c]) if self.union[c] > 0 else np.nan
     95 
     96 class BackgroundIOU(IOU): nm = 'background'



ipdb>  p c


0


ipdb>  CLASS_INDEX[self.nm]


0


ipdb>  self.inter[c]


*** KeyError: 0


ipdb>  self.inter


{}


ipdb>  q
